## 20220422 Mov2Img

動画ファイルから画像ファイルを作り出すスクリプト  
はじめに、動画の情報を読み取る。  
  
その後、save_all_frames、save_frame、save_frame_range、save_frame_range_secの4種類の  
関数を定義し、実行できる。  
それぞれの関数の定義に合わせて、動画から画像に変換する。

[参考にしたページ](https://note.nkmk.me/python-opencv-video-to-still-image/)  
[サンプル動画](http://www.ss-dc.com/tokusyu/tokusyu56.html)：
NHKクリエイティブライブラリから動画を拝借。

In [1]:
####################
# Libraries
####################
import cv2
import os

In [2]:
####################
# Config
####################
class Config:
    video_path = '../input/sample_video.mp4'
    dir_path = '../output'
    result_path = '../output/sample_frame.jpg'
    basename = 'sample'
    ext = 'jpg'
    frame_num = 20
    start_frame = 0
    stop_frame = 700
    step_frame = 20
    start_sec = 0
    stop_sec = 100
    step_sec = 2.5
    
cfg = Config()

configで設定した動画パスから動画を読み込んで表示してみる。

In [3]:
# 動画の読み込み
cap = cv2.VideoCapture(cfg.video_path)

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', gray)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


動画のフレーム長さ、秒数を取得する。

In [6]:
cap = cv2.VideoCapture(cfg.video_path)                # 動画を読み込む
video_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT) # フレーム数を取得する
video_fps = cap.get(cv2.CAP_PROP_FPS)                 # フレームレートを取得する
video_len_sec = video_frame_count / video_fps         # 長さ（秒）を計算する
print(f"動画のフレーム長: {video_frame_count}")
print(f"動画の長さ: {video_len_sec:.2f}sec")

動画のフレーム長: 807.0
動画の長さ: 26.93sec


# Main

##### すべてのフレームを画像ファイルとして保存
---

In [ ]:
# すべてのフレームを画像ファイルとして保存
def save_all_frames(video_path, dir_path, basename, ext='jpg'):
    """指定フォルダ内にあるすべての動画ファイルを画像ファイルに変換する。

    Args:
        video_path (Path): 動画ファイルのPath
        dir_path (Path): 保存先フォルダ
        basename (str): 保存ファイル名
        ext (str, optional): 保存する画像の拡張子. Defaults to 'jpg'.
    """
    # OpenCVのVideoCaptureで動画を読み込み
    cap = cv2.VideoCapture(video_path)
    # 動画ファイルが開いていたら実行をやめる。
    if not cap.isOpened():
        return
    
    # 出力先フォルダの作成
    os.makedirs(dir_path, exist_ok=True) 
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0

    while True:
        
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
            n += 1
        else:
            return

In [ ]:
# 実行(普通に実行すると、けっこうな量のjpgが生成されるので、要注意)
if False:
    save_all_frames(
        video_path = cfg.video_path, 
        dir_path = cfg.dir_path, 
        basename = cfg.basename, 
        ext = cfg.ext,
        )

##### 任意のフレームを指定して画像ファイルとして保存
---

In [ ]:
# 単独のフレームを指定
def save_frame(video_path, frame_num, result_path):
    """任意のフレームを指定して保存

    Args:
        video_path (Path): 動画ファイルのパス
        frame_num (int): フレームNo.
        result_path (Path): 保存ファイルのパス
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

    ret, frame = cap.read()

    if ret:
        cv2.imwrite(result_path, frame)

In [ ]:
# 実行
if False:
    save_frame(
        video_path = cfg.video_path, 
        frame_num = cfg.frame_num, 
        result_path = cfg.result_path, 
        )

In [ ]:
# 任意の範囲のフレームを指定
def save_frame_range(video_path, start_frame, stop_frame, step_frame,
                     dir_path, basename, ext='jpg'):
    """指定の範囲のフレームの画像を保存

    Args:
        video_path (Path): 動画ファイルのパス
        start_frame (int): 開始フレームNo.
        stop_frame (int): 終了フレームNo.
        step_frame (int): フレームのステップ数
        dir_path (Path): 保存先のパス
        basename (str): ファイルのベース名
        ext (str, optional): 画像の拡張子. Defaults to 'jpg'.
    """
    # 画像の読み込み
    cap = cv2.VideoCapture(video_path)
    # 画像が開いていたら処理終了
    if not cap.isOpened():
        return

    # 保存フォルダの作成
    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    for n in range(start_frame, stop_frame, step_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
        else:
            return

In [ ]:
# 実行
if False:
    save_frame_range(
        video_path = cfg.video_path, 
        start_frame = cfg.start_frame, 
        stop_frame = cfg.stop_frame, 
        step_frame = cfg.step_frame,
        dir_path = cfg.dir_path, 
        basename = cfg.basename, 
        ext='jpg'
        )

In [ ]:
# 時間（秒数）で指定 (time[sec] = frame / FPS)
def save_frame_range_sec(video_path, start_sec, stop_sec, step_sec,
                         dir_path, basename, ext='jpg'):
    """切り出す時間を指定して画像にする。

    Args:
        video_path (Path): 動画ファイルのパス
        start_sec (float): 開始の時刻
        stop_sec (float): 終了の時刻
        step_sec (float): 時間ステップ
        dir_path (Path): 保存先のパス
        basename (str): 保存ファイルのベース名
        ext (str, optional): 画像の拡張子. Defaults to 'jpg'.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps_inv = 1 / fps

    sec = start_sec
    while sec < stop_sec:
        n = round(fps * sec)
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite(
                '{}_{}_{:.2f}.{}'.format(
                    base_path, str(n).zfill(digit), n * fps_inv, ext
                ),
                frame
            )
        else:
            return
        sec += step_sec

In [ ]:
# 実行
if True:
    save_frame_range_sec(
        video_path = cfg.video_path, 
        start_sec = cfg.start_sec, 
        stop_sec = cfg.stop_sec, 
        step_sec = cfg.step_sec,
        dir_path = cfg.dir_path, 
        basename = cfg.basename, 
        ext='jpg'
        )

##### 動画を再生しながらキーボード押下で保存
---

In [ ]:
def save_frame_play(video_path, dir_path, basename, ext='jpg', delay=1, window_name='frame'):
    """動画を再生しながら、キーボード押下で保存

    Args:
        video_path (Path): 動画のパス
        dir_path (Path): 保存先フォルダのパス
        basename (str): ベース名
        ext (str, optional): 画像の拡張子. Defaults to 'jpg'.
        delay (int, optional): 処理のディレイ. Defaults to 1.
        window_name (str, optional): ウィンドウの名前. Defaults to 'frame'.
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0
    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow(window_name, frame)
            key = cv2.waitKey(delay) & 0xFF
            if key == ord('c'):
                cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
            elif key == ord('q'):
                break
            n += 1
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            n = 0

    cv2.destroyWindow(window_name)

In [ ]:
# 実行
if False:
    save_frame_play(
        video_path = cfg.video_path, 
        dir_path = cfg.dir_path, 
        basename = cfg.basename, 
        ext='jpg', 
        delay=1, 
        window_name='frame'
        )